## Homework

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-12 10:28:10--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 108.177.98.102, 108.177.98.101, 108.177.98.138, ...
Connecting to docs.google.com (docs.google.com)|108.177.98.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/medpnpdaoe253hjj9fi3psg394uvjgot/1652351250000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-12 10:28:12--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/medpnpdaoe253hjj9fi3psg394uvjgot/1652351250000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.goog

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
#from sklearn import tree, metrics, model_selection
import matplotlib.pyplot as plt
from collections import Counter
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"]).drop(0, axis = 0).drop(['id'], axis = 1).reset_index(drop = True)

In [6]:
df.head(4)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
1,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,...,13,6,1,8,18,0,1,52,0,17000000
2,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
3,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,2.0,...,3,1,0,5,8,0,1,34,5,7900000


In [7]:
drop_columns = ['timestamp'] #id уже удален

cat_columns = ['product_type', 'material', 'state', 'sub_area', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion',
    'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'ID_metro', 'ID_railroad_station_walk',
    'ID_railroad_station_avto', 'water_1line', 'ID_big_road1', 'big_road1_1line', 'ID_big_road2', 'railroad_1line', 'ID_railroad_terminal', 'ID_bus_terminal','ecology']

num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

*Поработаем теперь с заполнением данных*

В процессе подготовки данных нашлась ошибка в годе постройки:

In [8]:
print(df.iloc[df['build_year'].argmax()]['build_year'])
if df.iloc[df['build_year'].argmax()]['build_year'] > 2030:
  df.at[df['build_year'].argmax(), 'build_year'] = 2005

20052009.0


Разберемся с площадью и этажностью, там нельзя просто вставить среднее, т.к. жилая площадь <= полной и с этажами аналогично.

In [9]:
df['full_sq'] = df.apply(lambda row: row['life_sq'] if row['full_sq'] < row['life_sq'] else row['full_sq'], axis = 1) #чтобы полная площадь была не меньше жилой

df = df[df['full_sq'] > 5] #странная какая-то площадь 5 кв.м.

df_squares = df[['full_sq', 'life_sq']].dropna(how = 'any')
df_squares = df_squares[df_squares['life_sq'] > 0]
df_squares_fraq = (df_squares['life_sq']/df_squares['full_sq']).mean() #для тех, кто не указал жилую, умножим полную на среднее отношение площадей
df['life_sq'] = df.apply(lambda row: row['full_sq']*df_squares_fraq if np.isnan(row['life_sq']) else row['life_sq'], axis = 1) 

In [10]:
df['floor'].fillna(df['floor'].median(), inplace =True)
df['max_floor'] = df.apply(lambda row: row['floor'] if row['max_floor'] < row['floor'] else row['max_floor'], axis = 1)
df['max_floor'] = df.apply(lambda row: row['floor'] if np.isnan(row['max_floor']) else row['max_floor'], axis = 1)

df = df[(df['floor'] > 0) & (df['max_floor'] > 0)]

Устраним проблемы с числом комнат:

In [11]:
df.head(3)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,2012-10-04,64.0,64.0,16.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
1,2014-02-05,83.0,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,3.0,...,13,6,1,8,18,0,1,52,0,17000000
2,2012-07-26,71.0,49.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000


In [12]:
df_rooms = df[['full_sq', 'num_room']].dropna(how = 'any')
df_rooms = df_rooms[df_rooms['num_room'] > 0]
mean_room_square = (df_rooms['full_sq']/df_rooms['num_room']).mean()
df['num_room'] = df.apply(lambda row: row['full_sq']/mean_room_square if np.isnan(row['num_room']) else row['num_room'], axis = 1)
df['num_room'] = df.apply(lambda row: row['full_sq']/mean_room_square if row['num_room'] == 0 else row['num_room'], axis = 1)

Также пусть год постройки заполняется медианой, а остальные числовые признаки - средним


In [13]:
df['build_year'].fillna(df['build_year'].dropna().median(), inplace = True)

In [14]:
for col in num_columns:
  df[col].fillna(df[col].mean(), inplace = True)

Категориальные признаки заполним модой:

In [15]:
if df.isna().sum().sum() > 0:
  for col in cat_columns:
    df[col].fillna(df[col].mode()[0], inplace = True)
    #df[col].fillna('nan', inplace = True)

In [16]:
df = pd.get_dummies(df)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [17]:
X = df.drop(['price'], axis = 1).drop(drop_columns, axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=False, random_state=42)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:
```
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))
```

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [ ]:
tree_reg = DecisionTreeRegressor(min_samples_leaf=50, random_state=42)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(min_samples_leaf=50, random_state=42)

Проверьте качество на отложенной выборке.

In [ ]:
y_pred = tree_reg.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

3036916.698234333

### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [ ]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

Добавьте следующие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [ ]:
df['month'] = df.timestamp.dt.month
df['dayofweek'] = df.timestamp.dt.dayofweek
df['relative_floor'] = df['floor']/df['max_floor']
df['kitchen_vs_flat'] = df['kitch_sq']/df['full_sq']

df['room_sq'] = df['full_sq']/df['num_room']

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [ ]:
X = df.drop(['price'], axis = 1).drop(drop_columns, axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state=42)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

Посмотрим сначала на обычные деревья и бэггинг с ними.

In [ ]:
tree_reg = DecisionTreeRegressor(min_samples_leaf=50, random_state=42)
tree_reg.fit(X_train, y_train)

y_pred = tree_reg.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

3185270.2662596875

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
bagging_reg = BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=42, max_features='sqrt'), n_estimators=50, random_state=42)
bagging_reg.fit(X_train, y_train)

y_pred = bagging_reg.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2990460.0186792766

Теперь на случайный лес, и попробуем подобрать оптимальные параметры.

In [ ]:
random_forest_reg = RandomForestRegressor(random_state=42, n_jobs = -1)
random_forest_reg.fit(X_train, y_train)

y_pred = random_forest_reg.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2798363.055633215

In [18]:
#!pip install optuna
import optuna

In [ ]:
def objective(trial):

  n_estimators = trial.suggest_int("n_estimators", 100, 500, 400)
  max_depth = trial.suggest_categorical("max_depth", [ 50, None])

  forest_model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = "auto") #auto показывал результаты лучше sqrt
  forest_model.fit(X_train, y_train)
  y_pred = forest_model.predict(X_test)
 
  return mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
study = optuna.create_study(study_name="RF")
study.optimize(objective, n_trials=6, n_jobs = -1)

[I 2022-05-11 14:12:17,566] A new study created in memory with name: RF
[I 2022-05-11 14:22:49,881] Trial 4 finished with value: 2741491.8781349673 and parameters: {'n_estimators': 100, 'max_depth': None, 'max_features': 'auto'}. Best is trial 4 with value: 2741491.8781349673.
[I 2022-05-11 14:58:00,487] Trial 3 finished with value: 2728086.7452210127 and parameters: {'n_estimators': 500, 'max_depth': 50, 'max_features': 'auto'}. Best is trial 3 with value: 2728086.7452210127.
[I 2022-05-11 15:01:29,485] Trial 0 finished with value: 2731462.5199570744 and parameters: {'n_estimators': 500, 'max_depth': 50}. Best is trial 0 with value: 2731462.5199570744.
[I 2022-05-11 15:01:30,874] Trial 1 finished with value: 2745926.9012746997 and parameters: {'n_estimators': 500, 'max_depth': 50}. Best is trial 0 with value: 2731462.5199570744.
[I 2022-05-11 15:07:50,247] Trial 2 finished with value: 2744020.2930890736 and parameters: {'n_estimators': 100, 'max_depth': 50}. Best is trial 0 with value

In [ ]:
best_forest_params = study.best_params
best_forest_model = RandomForestRegressor(**best_forest_params, random_state=42, n_jobs = -1)

best_forest_model.fit(X_train, y_train)

y_pred = best_forest_model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2747021.6759853032

Теперь посмотрим на Catboost, поищем гиперпараметры там с помощью Optuna. \
Разработчики Catboost рекомендуют не использовать one-hot coding.

In [19]:
#!pip install catboost
from catboost import CatBoostRegressor


In [19]:
cat_model = CatBoostRegressor(verbose = False, random_state = 42)# , max_bin=3
cat_model.fit(X_train, y_train)
y_pred = cat_model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2677476.2449656893

In [20]:
def objective(trial):
  l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1, 5)
  learning_rate = trial.suggest_float("learning_rate", 0.00001, 1)
  cat_model = CatBoostRegressor(learning_rate = learning_rate, l2_leaf_reg=l2_leaf_reg, iterations = 500, verbose = False)
  cat_model.fit(X_train, y_train)
  y_pred = cat_model.predict(X_test)
 
  return mean_squared_error(y_test, y_pred, squared=False)

In [21]:
study = optuna.create_study(study_name="Catboost")
study.optimize(objective, n_trials=7, n_jobs = -1, show_progress_bar=True)

[I 2022-05-12 10:28:28,360] A new study created in memory with name: Catboost
Custom logger is already specified. Specify more than one logger at same time is not thread safe.[I 2022-05-12 10:29:23,018] Trial 1 finished with value: 2879907.7297702665 and parameters: {'l2_leaf_reg': 2.9477758669856393, 'learning_rate': 0.673888673719942}. Best is trial 1 with value: 2879907.7297702665.
[I 2022-05-12 10:29:32,745] Trial 0 finished with value: 2802931.2075351863 and parameters: {'l2_leaf_reg': 3.7875666331093503, 'learning_rate': 0.5745469743897773}. Best is trial 0 with value: 2802931.2075351863.
[I 2022-05-12 10:30:19,508] Trial 2 finished with value: 2799327.2907123826 and parameters: {'l2_leaf_reg': 2.8138125256704085, 'learning_rate': 0.40238338335727436}. Best is trial 2 with value: 2799327.2907123826.
[I 2022-05-12 10:30:34,560] Trial 3 finished with value: 2982013.9312236076 and parameters: {'l2_leaf_reg': 4.885438097630977, 'learning_rate': 0.947287057515265}. Best is trial 2 wit

In [22]:
best_catboost_params = study.best_params
best_catboost_model = CatBoostRegressor(**best_catboost_params, random_state = 42, verbose = False)

best_catboost_model.fit(X_train, y_train)

y_pred = best_catboost_model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

2785920.147387887

На основе имеющихся данных можно сделать вывод о том, что Catboost решает задачу регрессии лучше других рассмотренных моделей (RandomForest, DecisionTreeRegressor, Bagging + DecisionTrees)

### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [35]:
invest_train_X = X_train[X_train['product_type_Investment'] == 1].drop(['product_type_Investment'], axis = 1)
invest_train_Y = y_train[X_train['product_type_Investment'] == 1]
invest_test_X = X_test[X_test['product_type_Investment'] == 1].drop(['product_type_Investment'], axis = 1)
invest_test_Y = y_test[X_test['product_type_Investment'] == 1]

owner_train_X = X_train[X_train['product_type_OwnerOccupier'] == 1].drop(['product_type_OwnerOccupier'], axis = 1)
owner_train_Y = y_train[X_train['product_type_OwnerOccupier'] == 1]
owner_test_X = X_test[X_test['product_type_OwnerOccupier'] == 1].drop(['product_type_OwnerOccupier'], axis = 1)
owner_test_Y = y_test[X_test['product_type_OwnerOccupier'] == 1]

In [23]:
def product_type_model(invest_train_X, invest_train_Y, invest_test_X, invest_test_Y, 
                       owner_train_X, owner_train_Y, owner_test_X, owner_test_Y):


  cat_model_invest = CatBoostRegressor(verbose = False, random_state = 42)
  cat_model_invest.fit(invest_train_X, invest_train_Y)
  y_pred = cat_model_invest.predict(invest_test_X)
  invest_model_rmse = mean_squared_error(invest_test_Y, y_pred, squared=False)

  cat_model_owner = CatBoostRegressor(verbose = False, random_state = 42)
  cat_model_owner.fit(owner_train_X, owner_train_Y)
  y_pred = cat_model_owner.predict(owner_test_X)
  owner_model_rmse = mean_squared_error(owner_test_Y, y_pred, squared=False)

  print('invest_model RMSE: ', invest_model_rmse)
  print('owner_model RMSE: ', owner_model_rmse)

product_type_model(invest_train_X, invest_train_Y, invest_test_X, invest_test_Y, 
                       owner_train_X, owner_train_Y, owner_test_X, owner_test_Y)

invest_model RMSE:  2955807.9398104264
owner_model RMSE:  2007131.9837216274


### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [24]:
from sklearn.linear_model import HuberRegressor

In [40]:
def objective(trial):
  max_iter = trial.suggest_int("max_iter", 50, 500, 50)
  epsilon = trial.suggest_float("epsilon", 1, 3)
  alpha = trial.suggest_float("alpha", 0.00001, 0.01)
  warm_start = trial.suggest_categorical("warm_start", [True, False])
  fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
  tol = trial.suggest_float("epsilon", 1e-7, 1e-2)
  Huber = HuberRegressor(max_iter = max_iter, epsilon=epsilon, alpha = alpha, warm_start = warm_start, 
                         fit_intercept = fit_intercept, tol = tol)
  Huber.fit(invest_train_X, invest_train_Y)
  y_pred = Huber.predict(invest_test_X)
 
  return mean_squared_error(invest_test_Y, y_pred, squared=False)

In [42]:
study = optuna.create_study(study_name="Huber")
study.optimize(objective, n_trials=20, n_jobs = -1, show_progress_bar=True)

[I 2022-05-12 11:12:23,665] A new study created in memory with name: Huber
[I 2022-05-12 11:12:37,065] Trial 1 finished with value: 5136132.807252785 and parameters: {'max_iter': 150, 'epsilon': 2.7646480927739745, 'alpha': 0.0038048542170067558, 'warm_start': True, 'fit_intercept': False}. Best is trial 1 with value: 5136132.807252785.
[I 2022-05-12 11:12:51,762] Trial 2 finished with value: 5133188.384662846 and parameters: {'max_iter': 250, 'epsilon': 1.8805146105449237, 'alpha': 0.009778367290028954, 'warm_start': False, 'fit_intercept': True}. Best is trial 2 with value: 5133188.384662846.
[I 2022-05-12 11:12:53,646] Trial 0 finished with value: 5080059.56994567 and parameters: {'max_iter': 450, 'epsilon': 2.451525832784362, 'alpha': 0.008339739306133057, 'warm_start': False, 'fit_intercept': False}. Best is trial 0 with value: 5080059.56994567.
[I 2022-05-12 11:13:05,650] Trial 3 finished with value: 5135036.225962242 and parameters: {'max_iter': 250, 'epsilon': 2.985768426014835

In [46]:
def product_type_model(invest_train_X, invest_train_Y, invest_test_X, invest_test_Y, 
                       owner_train_X, owner_train_Y, owner_test_X, owner_test_Y):
  cat_model_invest = CatBoostRegressor(verbose = False, random_state = 42)
  cat_model_invest.fit(invest_train_X, invest_train_Y)
  y_pred_cat_model = cat_model_invest.predict(invest_test_X)
  invest_cat_model_rmse = mean_squared_error(invest_test_Y, y_pred_cat_model, squared=False)

  huber_model_invest = HuberRegressor(**study.best_params)
  huber_model_invest.fit(invest_train_X, invest_train_Y)
  y_pred_huber_model = huber_model_invest.predict(invest_test_X)
  invest_huber_model_rmse = mean_squared_error(invest_test_Y, y_pred_huber_model, squared=False)


  w1, w2 = 0.8, 0.2
  invest_model_rmse = mean_squared_error(invest_test_Y, y_pred_cat_model*w1 + y_pred_huber_model*w2, squared=False)


  cat_model_owner = CatBoostRegressor(**best_catboost_params, verbose = False, random_state = 42)
  cat_model_owner.fit(owner_train_X, owner_train_Y)
  y_pred = cat_model_owner.predict(owner_test_X)
  owner_model_rmse = mean_squared_error(owner_test_Y, y_pred, squared=False)


  print('Huber invest_model RMSE: ', invest_huber_model_rmse)
  print('Cat invest_model RMSE: ', invest_cat_model_rmse)
  print('difficult invest_model RMSE: ', invest_model_rmse)

  print('owner_model RMSE: ', owner_model_rmse)

product_type_model(invest_train_X, invest_train_Y, invest_test_X, invest_test_Y, 
                       owner_train_X, owner_train_Y, owner_test_X, owner_test_Y)

Huber invest_model RMSE:  5072532.342822795
Cat invest_model RMSE:  2955807.9398104264
difficult invest_model RMSE:  3026774.963163435
owner_model RMSE:  2121677.922824517


Из-за плохого результата Huber-регрессора суммарно результат как-то не улучшается